In [1]:
import argparse
import os
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import cv2
import models

def predict(img):
    i = 0
    tf.reset_default_graph()
    model_data_path = "NYU_FCRN.ckpt"
    
    height = 228
    width = 304
    channels = 3
    batch_size = 1
    
    input_node = tf.placeholder(tf.float32, shape=(None, height, width, channels))

    net = models.ResNet50UpProj({'data': input_node}, batch_size, 1, False)
        
    with tf.Session() as sess:
        saver = tf.train.Saver()     
        saver.restore(sess, model_data_path)
        img = img.resize([width,height], Image.ANTIALIAS)
        img = np.array(img).astype('float32')
        img = np.expand_dims(np.asarray(img), axis = 0)
        pred = sess.run(net.get_output(), feed_dict={input_node: img})
        
        return pred[0,:,:,:]

In [2]:
%matplotlib inline
import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import  confusion_matrix
from sklearn.model_selection import train_test_split
import time
from datetime import timedelta
import math

Using TensorFlow backend.


In [ ]:
import os
import h5py
import natsort
from os import listdir
import cv2
import pandas as pd
import numpy as np
from PIL import Image
directory1 = '/picsDB/'
directory2 = '/depthDB/'
dirs = natsort.natsorted(os.listdir( directory1 ))
data = pd.read_csv("DB.csv")

In [ ]:
import random
q = random.sample(range(0, 11500), 11500)

In [ ]:
with h5py.File('ARdataset.h5', 'a') as hdf:
    x_train = hdf.create_dataset('x_train', (10000,224,224,4),
                            dtype='float32', chunks=(10**3,224,224,4))
    y_train = hdf.create_dataset('y_train', (10000,12),
                            dtype='float32', chunks=(10**3,12))
    x_test = hdf.create_dataset('x_test', (1500,224,224,4),
                            dtype='float32', chunks=(10**3,224,224,4))
    y_test = hdf.create_dataset('y_test', (1500,12),
                            dtype='float32', chunks=(10**3,12))
    with h5py.File('DBrgbd.h5', 'a') as hd:
        for i in range(0,10000):
            x_train[i]=hd['x_train'][q[i]]
            y_train[i]=(hd['y_train'][q[i]])
            if(i%1000==0):
                print(i)
        for i in range(10000,11500):
            x_train[11499-i]=hd['x_train'][q[i]]
            y_train[11499-i]=hd['y_train'][q[i]]
            if(i%1000==0):
                print(y_train[11499-i])

In [3]:
import h5py

[-0.06        0.78        0.68        1.         -0.33333334  0.46
  0.34        1.          0.18        0.6666667  -0.06666667  0.        ]


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with h5py.File('DBrgbd.h5', 'r') as hdf:
    X, Y =  hdf['x_train'], hdf['y_train']
    model = Sequential()
    model.add(Conv2D(1, (5, 5), input_shape=(224, 224, 4),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(1, (3, 3), input_shape=(112, 112, 4),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(1, (3, 3), input_shape=(56, 56, 4),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(1, (3, 3), input_shape=(28, 28, 4),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(1, (3, 3), input_shape=(14, 14, 4),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(500,activation='tanh'))
    model.add(Dense(12,activation='tanh'))
    sgd = optimizers.SGD(lr=0.01)
    model.compile(loss='mean_squared_error',
              optimizer=sgd,
              metrics=['accuracy'])
    model.summary()
    model.fit(X,Y,shuffle="batch",batch_size=100,epochs=10,verbose=1,)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 1)       101       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 1)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 1)       10        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 1)         10        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 1)         10        
__________

In [5]:
import PIL
img = Image.open('1.jpg')
depthMap = predict(img)

INFO:tensorflow:Restoring parameters from NYU_FCRN.ckpt


In [6]:
import cv2
img = img.resize([160, 128], Image.ANTIALIAS)
data = cv2.resize(np.append(img,depthMap,axis=2),(224,224))
data=np.expand_dims(data,axis=0)
np.shape(data)

(1, 224, 224, 4)

In [7]:
from keras.models import model_from_json
with open('model_arch.json','r') as f:
    model = model_from_json(f.read())
model.load_weights('model_weights.h5')

In [8]:
model.predict(data)*(15,15,15,1,15,15,15,1,15,15,15,1)

array([[ 5.48599586, 11.96957588,  7.29419008,  0.60981941, -0.39890623,
         0.67384344, -7.46860966,  0.83537441,  1.03508379, 10.70313692,
        -4.6847257 ,  0.8592602 ]])